# Challenge 4 - Solution - Azure Machine Learning Pipelines

A common scenario when using machine learning components is to have a data workflow that includes the following steps:

- Preparing/preprocessing a given dataset for training, followed by
- Training a machine learning model on this data, and then
- Deploying this trained model in a separate environment, and finally
- Running a batch scoring task on another data set, using the trained model.

Azure's Machine Learning pipelines give you a way to combine multiple steps like these into one configurable workflow, so that multiple agents/users can share and/or reuse this workflow. Machine learning pipelines thus provide a consistent, reproducible mechanism for building, evaluating, deploying, and running ML systems.

To get more information about Azure machine learning pipelines, please read our [Azure Machine Learning Pipelines](https://aka.ms/pl-concept) overview, or the [readme article](https://aka.ms/pl-readme).

In this notebook, we provide a gentle introduction to Azure machine learning pipelines. We build a pipeline that runs jobs unattended on different compute clusters; in this notebook, you'll see how to use the basic Azure ML SDK APIs for constructing this pipeline.

## 1. Import Azure ML Python Python SDK

In [ ]:
import azureml.core
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
# Pipeline-specific SDK imports
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

print("Pipeline SDK-specific imports completed")

In [ ]:
# Additional libraries
import os

from azureml.core import Experiment, Datastore

## 2. Authentication and initializing Azure Machine Learning Workspace

As a first step you have to authenticate against the Azure [Machine Learning Workspace](https://ml.azure.com/). This can be achieved in different ways:

1. **Interactive Login Authentication:** The interactive authentication is suitable for local experimentation on your own computer.
2. **Azure CLI Authentication:** Azure CLI authentication is suitable if you are already using Azure CLI for managing Azure resources, and want to sign in only once.
3. **Managed Service Identity (MSI) Authentication:** The MSI authentication is suitable for automated workflows, for example as part of Azure Devops build.
4. **Service Principal Authentication:** The Service Principal authentication is suitable for automated workflows, for example as part of Azure Devops build.

For now, we will use the interactive authentication, which is the default mode when using Azure ML SDK. When you connect to your workspace using `Workspace.from_config`, you will get an interactive login dialog.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

After we logged in, we can print the Worspace details.

**TASK**: Print the workspace details below. See here for the workspace object reference: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py

In [ ]:
print("Workspace name: " + ws.name, 
      "Azure region: " + ws.location, 
      "Subscription id: " + ws.subscription_id, 
      "Resource group: " + ws.resource_group, sep = '\n')

## 3. Upload and register data

Every workspace comes with a default [datastore](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data) (and you can register more) which is backed by the Azure blob storage account associated with the workspace. We can use it to transfer data from local to the cloud, and create Dataset from it. We will now upload the Iris data to the default datastore (blob) within your workspace.

By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred.

In [ ]:
# List all datastores registered in the current workspace
datastores = ws.datastores
for name, datastore in datastores.items():
    print(name, datastore.datastore_type)

For this challenge we will use the [default datastore](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-access-data#get-datastores-from-your-workspace) that comes with the Azure Machine Learning Workspace.

**TASK**: Retrieve the default datastore for this workspace.

Hint: Same link as in the previous hint.

In [ ]:
# get the default datastore
datastore = ws.get_default_datastore()
print(datastore.name, datastore.datastore_type, datastore.account_name, datastore.container_name, sep="\n")

**TASK**: Upload the file `./train-dataset/iris.csv` to the target path `train-dataset/tabular/` on the default datastore.

Hint: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.azure_storage_datastore.azureblobdatastore

In [ ]:
# Here we are reusing the def_blob_store object we obtained earlier
def_blob_store.upload_files(files=[ os.path.join('.', 'train_dataset', '20news.pkl') ],
                            target_path='train_dataset/files/20newsgroups',
                            overwrite=True,
                            show_progress = True)

Now we will register a dataset in the Azure Machine Learning Workspace as a file dataset. A file dataset can be mounted to the compute engine. When you mount a file system, you attach that file system to a directory (mount point) and make it available to the system. Because mounting load files at the time of processing, it is usually faster than download.
Note: mounting is only available for Linux-based compute (DSVM/VM, AMLCompute, HDInsights).

In [ ]:
from azureml.core import Dataset

file_dataset = Dataset.File.from_files(path = [(datastore, 'train_dataset/image_dataset/hymenoptera_data')])
file_dataset = file_dataset.register(workspace=ws,
                                     name='hymenoptera_data',
                                     description='hymenoptera training dataset',
                                     create_new_version = True)

file_dataset.to_path()

# WIP!!!